<a href="https://colab.research.google.com/github/monuminu/affine_hackathon/blob/master/table_detection_using_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir table_detection

In [2]:
cd table_detection

/content/table_detection


In [37]:
!mkdir data/tfdata/classes-table

mkdir: cannot create directory ‘data/tfdata/classes-table’: No such file or directory


In [0]:
!rm 'config.yml'

In [49]:
from google.colab import files
files.upload()

Saving config.yml to config.yml


{'config.yml': b'train:\r\n  # Name used to identify the run. Data inside `job_dir` will be stored under\r\n  # `run_name`.\r\n  run_name: table-area-detection-0.1\r\n  # Base directory in which model checkpoints & summaries (for Tensorboard) will\r\n  # be saved.\r\n  job_dir: jobs/\r\n  save_checkpoint_secs: 10\r\n  save_summaries_secs: 10\r\n\r\n  # Number of epochs (complete dataset batches) to run.\r\n  num_epochs: 10\r\n\r\ndataset:\r\n  type: object_detection\r\n  # From which directory to read the dataset.\r\n  dir: tfdata/\r\n  image_preprocessing:\r\n    min_size: 600\r\n    max_size: 1024\r\n  data_augmentation:\r\n    - flip:\r\n        left_right: True\r\n        up_down: True\r\n        prob: 0.5\r\n\r\nmodel:\r\n  type: fasterrcnn\r\n  network:\r\n    # Total number of classes to predict.\r\n    num_classes: 1'}

In [0]:
!mv val.csv data/

In [20]:
import os
import cv2
import pandas as pd

root_dir = os.getcwd()
file_list = ['train.csv', 'val.csv']
image_source_dir = os.path.join(root_dir, 'data/images/')
data_root = os.path.join(root_dir, 'data')
for file in file_list:
    
    image_target_dir = os.path.join(data_root, file.split(".")[0])
    if not os.path.exists(image_target_dir):
      os.mkdir(image_target_dir)
    # read list of image files to process from file
    image_list = pd.read_csv(os.path.join(data_root, file), header=None)[0]
    
    print("Start preprocessing images")
    for image in image_list:
        # open image file
        img = cv2.imread(os.path.join(image_source_dir, image))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # perform transformations on image
        b = cv2.distanceTransform(img, distanceType=cv2.DIST_L2, maskSize=5)
        g = cv2.distanceTransform(img, distanceType=cv2.DIST_L1, maskSize=5)
        r = cv2.distanceTransform(img, distanceType=cv2.DIST_C, maskSize=5)
        
        # merge the transformed channels back to an image
        transformed_image = cv2.merge((b, g, r))
        target_file = os.path.join(image_target_dir, image)
        #print("Writing target file {}".format(target_file))
        cv2.imwrite(target_file, transformed_image)
print('Successfully processed all the images !!')        

Start preprocessing images
Start preprocessing images
Successfully processed all the images !!


In [22]:
!pip install luminoth

    100% |████████████████████████████████| 225kB 5.3MB/s 
    100% |████████████████████████████████| 71kB 5.3MB/s 
    100% |████████████████████████████████| 92kB 5.6MB/s 
    100% |████████████████████████████████| 624kB 7.0MB/s 
    100% |████████████████████████████████| 2.3MB 7.8MB/s 
    100% |████████████████████████████████| 5.8MB 4.8MB/s 
  Running setup.py bdist_wheel for easydict ... - done
  Stored in directory: /root/.cache/pip/wheels/3f/f0/7c/1ec164f1dc11a6e0ef58463f5e25e83d9c6f661e2e8e70bcef
Successfully built easydict


In [30]:
!lumi dataset transform --type csv --data-dir data/ --output-dir tfdata/ --split train --split val --only-classes=table

INFO:tensorflow:Saving split "train" in output_dir = tfdata/
  [####################################]  100%             
INFO:tensorflow:Saved 338 records to "tfdata/train.tfrecords"
INFO:tensorflow:Composition per class (train):
INFO:tensorflow:	table: 418
INFO:tensorflow:Saving split "val" in output_dir = tfdata/
  [####################################]  100%
INFO:tensorflow:Saved 65 records to "tfdata/val.tfrecords"
INFO:tensorflow:Composition per class (val):
INFO:tensorflow:	table: 100


In [62]:
!lumi train -c config.yml

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.
Instructions for updating:
Create a `

In [64]:
!lumi checkpoint create config.yml

Creating checkpoint for given configuration...
Checkpoint 336265de5d17 created successfully.


In [65]:
!lumi checkpoint list

|           id |                  name |       alias | source |         status |
| 685f3e04752c |                       |             |  local |          LOCAL |
| 336265de5d17 |                       |             |  local |          LOCAL |


In [66]:
!lumi checkpoint download 336265de5d17

Checkpoint is not remote. If you intended to download a remote checkpoint and used an alias, try using the id directly.


In [0]:
from google.colab import files
files.download('336265de5d17.tar')

In [68]:
!lumi checkpoint export 336265de5d17

Checkpoint 336265de5d17 exported successfully.


In [57]:
!ls

685f3e04752c.tar  config.yml  data  data.zip  jobs  tfdata


In [59]:
!ls job/

ls: cannot access 'job/': No such file or directory


In [0]:
result = !lumi predict --checkpoint 336265de5d17 data/val/9541_023.png

In [78]:
dict(result[-1])

ValueError: ignored

In [0]:
lumi predict --checkpoint 336265de5d17 data/val/9541_028.png
lumi checkpoint import 336265de5d17.tar